## import

In [68]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import json
import os
from selenium.webdriver.common.keys import Keys
import regex as re
from bson.json_util import dumps
from datetime import datetime

## connect MongoDB

In [32]:
client = MongoClient('mongodb://localhost:27017/')
client.drop_database('simplize')
db = client['simplize']

## get drive

In [69]:
driver = webdriver.Chrome()
url = 'https://simplize.vn/co-phieu/nganh/tai-chinh'
driver.get(url)
sleep(2)

In [70]:
#lấy lĩnh vực
linh_vuc = driver.find_elements(By.XPATH,
     "//div[contains(@class,'simplize-row css-pmt33i')]")
#list chứa lĩnh vực
collection_list = []
#loop chạy từng lĩnh vực
for chay_linh_vuc in linh_vuc:
    data = chay_linh_vuc.text
    collection_name = data
    collection = db[collection_name]
    collection.insert_one({"linhvuc": data})
    collection_list.append(data)

print(collection_list)

# Tạo dic ma_list cho từng linh_vuc
ma_lists = []  # Danh sách chứa các ma_list của từng linh_vuc
#loop từng lĩnh vực
for index in range(len(linh_vuc)):
    # Cập nhật lại danh sách linh_vuc trước khi nhấp vào phần tử
    linh_vuc = driver.find_elements(By.XPATH, 
        "//div[contains(@class,'simplize-row css-pmt33i')]")
    linh_vuc[index].click() 
    sleep(2)
    
    # Lấy information 
    cac_ma = driver.find_elements(By.XPATH, 
                        "//div[contains(@class,'css-70qvj9')]")
    ma_list = []  # Tạo list để chứa các thông tin vừa lấy được

    for ma in cac_ma:
        ma_list.append(ma.text)  # add vào list

    print(f"Ma list for {collection_list[index]}: {ma_list}")
    print(len(ma_list))
    ma_lists.append(ma_list)  # add ma_list vào ma_lists

    driver.back()  # back trang trước để click linh vực tiếp theo
    sleep(2)  

['Tài chính ngân hàng', 'Chứng khoán và Ngân hàng đầu tư', 'Bảo hiểm']
Ma list for Tài chính ngân hàng: ['VCB', 'BID', 'CTG', 'TCB', 'VPB', 'MBB', 'ACB', 'LPB', 'HDB', 'STB', 'VIB', 'SSB', 'TPB', 'SHB', 'EIB', 'MSB', 'OCB', 'NAB', 'BAB', 'EVF', 'ABB', 'PGB', 'BVB', 'VBB', 'VAB', 'NVB', 'KLB', 'SGB', 'TIN']
29
Ma list for Chứng khoán và Ngân hàng đầu tư: ['SSI', 'VND', 'HCM', 'VCI', 'VIX', 'MBS', 'FTS', 'SHS', 'BSI', 'DSE', 'CTS', 'ORS', 'VDS', 'DSC', 'AGR', 'TVS', 'BVS', 'APG', 'VFS', 'PHS', 'AAS', 'TVB', 'TCI', 'EVS', 'ABW', 'SBS', 'IVS', 'BMS', 'APS', 'CSI']
30
Ma list for Bảo hiểm: ['BVH', 'PVI', 'BIC', 'VNR', 'MIG', 'BMI', 'PTI', 'PGI', 'PRE', 'ABI', 'AIC', 'BHI', 'BLI']
13


## cut list 

In [66]:
for i in ma_lists:
    print(len(i))

29
30
13


In [36]:
##remove list for testing

# to_remove1 = ('BID','CTG', 'TCB', 'VPB', 'MBB', 'ACB', 'LPB', 'HDB', 'STB', 'VIB', 'SSB', 'TPB', 'SHB', 'EIB', 'MSB', 'OCB', 'NAB', 'BAB', 'EVF', 'ABB', 'PGB', 'BVB', 'VBB', 'VAB', 'NVB', 'KLB', 'SGB','TIN')
# to_remove2 = ('SSI','VND','VCI', 'VIX', 'MBS', 'FTS', 'SHS', 'BSI', 'DSE', 'CTS', 'VDS', 'ORS', 'DSC', 'AGR', 'TVS', 'BVS', 'APG', 'VFS', 'PHS', 'AAS', 'TVB', 'TCI', 'EVS', 'ABW', 'SBS', 'IVS', 'BMS', 'APS','CSI')
# to_remove3 = ('PVI', 'VNR', 'BIC', 'MIG', 'BMI', 'PGI', 'PTI', 'PRE', 'ABI', 'AIC', 'BHI')

# # Xóa bớt ma trong các list 
# ma_lists[0] = [x for x in ma_lists[0] if x not in to_remove1]
# ma_lists[1] = [y for y in ma_lists[1] if y not in to_remove2]
# ma_lists[2] = [z for z in ma_lists[2] if z not in to_remove3]

In [37]:
print(ma_lists)

[['VCB', 'BID', 'CTG', 'TCB', 'VPB', 'MBB', 'ACB', 'LPB', 'HDB', 'STB', 'VIB', 'SSB', 'TPB', 'SHB', 'EIB', 'MSB', 'OCB', 'NAB', 'BAB', 'EVF', 'ABB', 'PGB', 'BVB', 'VBB', 'VAB', 'NVB', 'KLB', 'SGB', 'TIN'], ['SSI', 'VND', 'HCM', 'VCI', 'VIX', 'MBS', 'FTS', 'SHS', 'BSI', 'DSE', 'CTS', 'ORS', 'VDS', 'DSC', 'AGR', 'TVS', 'BVS', 'APG', 'VFS', 'PHS', 'AAS', 'TVB', 'TCI', 'EVS', 'ABW', 'SBS', 'IVS', 'BMS', 'APS', 'CSI'], ['BVH', 'PVI', 'BIC', 'VNR', 'MIG', 'BMI', 'PTI', 'PGI', 'PRE', 'ABI', 'AIC', 'BHI', 'BLI']]


In [38]:
ma_lists = [['VCB']]

## get price history

In [39]:
# loop qua từng list mã cổ phiếu trong ma_lists
for ma_list in ma_lists:
    for ma in ma_list:  # loop qua từng mã cổ phiếu trong sub_list
        # Tạo URL từ từng giá trị trong ma_list
        lich_su_gia_url = f"https://simplize.vn/co-phieu/{ma}/lich-su-gia"  
        driver.get(lich_su_gia_url)  
        sleep(2)
        body = driver.find_element(By.TAG_NAME, "body")
        for _ in range(10):  
            body.send_keys(Keys.END)
            sleep(2)
        ls_gia_list = []  #  list để chứa data giá

        
        for page_num in range(1, 3):  
            print(f"Mã: {ma} - Trang {page_num}")

            # get all data giá
            ls_gia = driver.find_elements(By.XPATH, 
                "//tr[contains(@class,'simplize-table-row simplize-table-row-level-0')]")

            for row in ls_gia:
                row_data = {
                    "date": row.find_element(By.XPATH, ".//td[1]").text,              
                    "opening_price": row.find_element(By.XPATH, ".//td[2]").text,     
                    "highest_price": row.find_element(By.XPATH, ".//td[3]").text,    
                    "lowest_price": row.find_element(By.XPATH, ".//td[4]").text,      
                    "closing_price": row.find_element(By.XPATH, ".//td[5]").text,     
                    "price_change": row.find_element(By.XPATH, ".//td[6]").text,       
                    "percent_change": row.find_element(By.XPATH, ".//td[7]").text,     
                    "volume": row.find_element(By.XPATH, ".//td[8]").text,             
                }
                ls_gia_list.append(row_data)  # add data vào 

            
            try:
                next_button = driver.find_element(By.XPATH, 
                            '//*[@id="phan-tich"]/div[2]/div/div/div[2]/div[1]/div/div[3]/ul/li[9]/div')
                next_button.click()  
                sleep(10)  
            except Exception as e:
                print(f"error {e}")
                break  

        # Lưu danh sách vào file JSON
        with open(f"{ma}_lich_su_gia.json", "w", encoding='utf-8') as json_file:
            # Lưu danh sách vào file JSON
            json.dump(ls_gia_list, json_file, ensure_ascii=False, indent=4)  
        # back về trang trước và lấy mã tiếp theo
        driver.back()

Mã: VCB - Trang 1
error Message: element click intercepted: Element <div class="simplize-pagination-item-link css-11we714">...</div> is not clickable at point (831, 6). Other element would receive the click: <div class="simplize-row simplize-row-space-between" style="row-gap: 0px;">...</div>
  (Session info: chrome=130.0.6723.71)
Stacktrace:
	GetHandleVerifier [0x00007FF7B9223AF5+28005]
	(No symbol) [0x00007FF7B91883F0]
	(No symbol) [0x00007FF7B902580A]
	(No symbol) [0x00007FF7B907D6CE]
	(No symbol) [0x00007FF7B907B16C]
	(No symbol) [0x00007FF7B9078628]
	(No symbol) [0x00007FF7B907785D]
	(No symbol) [0x00007FF7B906990E]
	(No symbol) [0x00007FF7B909BA3A]
	(No symbol) [0x00007FF7B9069246]
	(No symbol) [0x00007FF7B909BC50]
	(No symbol) [0x00007FF7B90BB8B3]
	(No symbol) [0x00007FF7B909B7E3]
	(No symbol) [0x00007FF7B90675C8]
	(No symbol) [0x00007FF7B9068731]
	GetHandleVerifier [0x00007FF7B951646D+3118813]
	GetHandleVerifier [0x00007FF7B9566CC0+3448624]
	GetHandleVerifier [0x00007FF7B955CF3D

## get business information

In [40]:
# creat list data business inf
ho_so_data_list = []

for ma_list in ma_lists:
    for ma in ma_list:  # loop từng mã cổ phiếu trong list
        # Tạo URL từ từng giá trị trong ho_so_data_list
        ho_so_url = f"https://simplize.vn/co-phieu/{ma}/ho-so-doanh-nghiep"  
        driver.get(ho_so_url)  
        sleep(2)
        
        # get thông tin
        percentage = driver.find_elements(By.XPATH, 
                                '//*[@class="css-xsfkf2"]//span//span')
        ho_so_list = []  # save list để chứa data

        for i in percentage:
            ho_so_list.append(i.text)  # add data in list

        # save data ho_so_list vào file JSON
        with open(f"{ma}_ho_so_doanh_nghiep.json", "w", encoding='utf-8') as json_file:
            json.dump(ho_so_list, json_file, ensure_ascii=False, indent=4)  


## update data

In [41]:
# loop từng phần tử trong ma_lists
for index, ma_list in enumerate(ma_lists):
    collection_name = collection_list[index]  # Lấy tên lĩnh vực tương ứng
    collection = db[collection_name]  # Truy cập vào collection tương ứng

    # Lặp qua từng mã cổ phiếu trong ma_list
    for ma in ma_list:
        json_file_path_lich_su_gia = f"{ma}_lich_su_gia.json"
        json_file_path_ho_so_doanh_nghiep = f"{ma}_ho_so_doanh_nghiep.json"

        # check file json "ho_so_doanh_nghiep"
        if os.path.exists(json_file_path_lich_su_gia):
            with open(json_file_path_lich_su_gia, "r", encoding='utf-8') as json_file:
                ls_gia_list = json.load(json_file)
        # check file json "ho_so_doanh_nghiep"
        if os.path.exists(json_file_path_ho_so_doanh_nghiep):
            with open(json_file_path_ho_so_doanh_nghiep, "r", encoding='utf-8') as json_file:
                ho_so_list = json.load(json_file)

        collection.update_one(
            {"linhvuc": collection_name},  # Tìm đối tượng có giá trị là collection_name
            {
                "$set": {
                    f"lich_su_gia.{ma}": ls_gia_list,  # update lịch sử giá
                    f"ho_so_doanh_nghiep.{ma}": ho_so_list  # update hồ sơ doanh nghiệp
                }
            },
            upsert=True  # Tạo mới nếu không tồn tại
        )

## close driver

In [42]:
driver.quit()

In [ ]:
collections = db.collection_list.find()

NameError: name 'db' is not defined

## close dbs

In [43]:
client.close()

In [44]:
#1 in ra toàn bộ

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']

# In toàn bộ dữ liệu trong từng collection
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    print(f"Dữ liệu trong collection '{collection_name}':")
    print(dumps(list(collection.find()), ensure_ascii=False, indent=4))


Dữ liệu trong collection 'Chứng khoán và Ngân hàng đầu tư':
[
    {
        "_id": {
            "$oid": "67225b086fb90585046cb327"
        },
        "linhvuc": "Chứng khoán và Ngân hàng đầu tư"
    }
]
Dữ liệu trong collection 'Bảo hiểm':
[
    {
        "_id": {
            "$oid": "67225b086fb90585046cb328"
        },
        "linhvuc": "Bảo hiểm"
    }
]
Dữ liệu trong collection 'Tài chính ngân hàng':
[
    {
        "_id": {
            "$oid": "67225b086fb90585046cb326"
        },
        "linhvuc": "Tài chính ngân hàng",
        "ho_so_doanh_nghiep": {
            "VCB": [
                "Cổ đông chiến lược 89.81%",
                "89.8%",
                "0%",
                "Quỹ đầu tư 19.01%",
                "15%",
                "2.81%",
                "0.89%",
                "0.2%",
                "0.08%",
                "0.02%",
                "0%",
                "Cổ đông thông thường -8.82%",
                "-8.82%"
            ]
        },
        "lich_su_

In [45]:
#2 in ra tên của tất cả các mã trong tài chính ngân hàng

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']

# Tên collection chứa dữ liệu tài chính
collection_name = "Tài chính ngân hàng"  # Thay bằng tên chính xác của collection
collection = db[collection_name]

# Tập hợp tất cả mã cổ phiếu từ các tài liệu trong collection
ma_cophieu_set = set()
for document in collection.find({}, {"lich_su_gia": 1}):
    ma_cophieu_set.update(document.get("lich_su_gia", {}).keys())

# In ra danh sách mã cổ phiếu
print("Danh sách các mã cổ phiếu:")
print("\n".join(ma_cophieu_set))


Danh sách các mã cổ phiếu:
VCB


In [46]:
#3 lấy dữ liệu của giá cổ phiếu của các mã trong Tài chính ngân hàng

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']

# Truy vấn dữ liệu từ collection "Tài chính ngân hàng"
collection = db['Tài chính ngân hàng']
cursor = collection.find({}, {"_id": 0, "lich_su_gia": 1})  # Lấy trường "lich_su_gia" của tất cả các mã

# In dữ liệu của giá cổ phiếu ra màn hình
for document in cursor:
    print(dumps(document, ensure_ascii=False, indent=4))


{
    "lich_su_gia": {
        "VCB": [
            {
                "date": "30/10/2024",
                "opening_price": "92,000",
                "highest_price": "92,400",
                "lowest_price": "91,600",
                "closing_price": "91,700",
                "price_change": "-300",
                "percent_change": "-0.33%",
                "volume": "626,500"
            },
            {
                "date": "29/10/2024",
                "opening_price": "92,100",
                "highest_price": "92,600",
                "lowest_price": "92,000",
                "closing_price": "92,000",
                "price_change": "-",
                "percent_change": "-",
                "volume": "1,297,600"
            },
            {
                "date": "28/10/2024",
                "opening_price": "91,500",
                "highest_price": "92,300",
                "lowest_price": "91,500",
                "closing_price": "92,000",
                "price_chan

In [47]:
#4 đếm số lượng data có trong tài chính ngân hàng

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db["Tài chính ngân hàng"]

# Biến đếm số lượng mã cổ phiếu và tổng số phần tử trong lịch sử giá
num_ma = 0
total_entries = 0

# Duyệt qua từng tài liệu trong collection và đếm số mã cổ phiếu cùng tổng số dữ liệu trong lịch sử giá
for document in collection.find({}, {"_id": 0, "lich_su_gia": 1}):
    lich_su_gia = document.get("lich_su_gia", {})
    num_ma += len(lich_su_gia)
    total_entries += sum(len(gia_data) for gia_data in lich_su_gia.values())  # Tính tổng số phần tử trong các mảng lịch sử giá

print(f"Số lượng mã cổ phiếu trong lịch sử giá: {num_ma}")
print(f"Tổng số phần tử trong các mảng lịch sử giá của các mã: {total_entries}")


Số lượng mã cổ phiếu trong lịch sử giá: 1
Tổng số phần tử trong các mảng lịch sử giá của các mã: 30


In [48]:
#5 đếm tổng volume của một mã bất kì 

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db["Tài chính ngân hàng"]

# Mã cổ phiếu cần tính tổng volume
ma_cophieu = "VCB"

# Truy vấn tài liệu chứa mã cổ phiếu "VCB" trong "lich_su_gia"
document = collection.find_one({f"lich_su_gia.{ma_cophieu}": {"$exists": True}}, {"lich_su_gia": 1})

# Tính tổng volume nếu mã cổ phiếu tồn tại
if document:
    total_volume = sum(int(record["volume"].replace(",", "")) for record in document["lich_su_gia"].get(ma_cophieu, []))
    print(f"Tổng volume của mã {ma_cophieu}: {total_volume}")
else:
    print(f"Không tìm thấy mã cổ phiếu {ma_cophieu} trong lịch sử giá.")


Tổng volume của mã VCB: 37808100


In [49]:
#6 lấy giá đóng cửa cao nhất trong một mã bất kì

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db["Tài chính ngân hàng"]

# Mã cổ phiếu cần tìm
ma_cophieu = "VCB"

# Truy vấn tài liệu chứa mã cổ phiếu trong "lich_su_gia"
document = collection.find_one({f"lich_su_gia.{ma_cophieu}": {"$exists": True}}, {"lich_su_gia": 1})

if document:
    lich_su_gia = document["lich_su_gia"].get(ma_cophieu, [])
    
    # Tìm giá đóng cửa lớn nhất
    try:
        max_closing_price = max(float(record["closing_price"].replace(",", "")) for record in lich_su_gia)
        print(f"Giá đóng cửa lớn nhất của mã {ma_cophieu}: {max_closing_price}")
    except ValueError:
        print(f"Không có dữ liệu giá đóng cửa hợp lệ cho mã {ma_cophieu}.")
else:
    print(f"Không tìm thấy mã cổ phiếu {ma_cophieu} trong lịch sử giá.")


Giá đóng cửa lớn nhất của mã VCB: 92800.0


In [50]:
from pymongo import MongoClient

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db["Tài chính ngân hàng"]

# Khởi tạo biến để lưu giá mở cửa lớn nhất
max_opening_price = float('-inf')
max_opening_stock = None

# Duyệt qua từng tài liệu trong collection
for document in collection.find({}, {"lich_su_gia": 1}):
    lich_su_gia = document.get("lich_su_gia", {})
    for ma_cophieu, records in lich_su_gia.items():
        for record in records:
            opening_price_str = record.get("opening_price", "")
            # Kiểm tra và bỏ qua các giá trị rỗng hoặc không hợp lệ
            if opening_price_str:
                try:
                    opening_price = float(opening_price_str.replace(",", ""))
                    if opening_price > max_opening_price:
                        max_opening_price = opening_price
                        max_opening_stock = ma_cophieu
                except ValueError:
                    # Bỏ qua các giá trị không thể chuyển đổi
                    continue

if max_opening_stock:
    print(f"Giá mở cửa lớn nhất: {max_opening_price} của mã cổ phiếu: {max_opening_stock}")
else:
    print("Không có dữ liệu giá mở cửa hợp lệ.")


Giá mở cửa lớn nhất: 92900.0 của mã cổ phiếu: VCB


In [51]:
#7 lấy giá mở của lớn nhất trong tất cả các mã

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Khởi tạo giá trị lớn nhất
max_opening_price, max_opening_stock = float('-inf'), None

# Duyệt qua từng tài liệu và tìm giá mở cửa lớn nhất
for document in collection.find({}, {"lich_su_gia": 1}):
    for ma_cophieu, records in document.get("lich_su_gia", {}).items():
        for record in records:
            opening_price_str = record.get("opening_price", "").replace(",", "")
            if opening_price_str:
                try:
                    opening_price = float(opening_price_str)
                    if opening_price > max_opening_price:
                        max_opening_price, max_opening_stock = opening_price, ma_cophieu
                except ValueError:
                    continue

# Kết quả
if max_opening_stock:
    print(f"Giá mở cửa lớn nhất: {max_opening_price} của mã cổ phiếu: {max_opening_stock}")
else:
    print("Không có dữ liệu giá mở cửa.")


Giá mở cửa lớn nhất: 92900.0 của mã cổ phiếu: VCB


In [52]:
#8 lấy % thay đổi nhỏ nhất của một mã bất kì

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Mã cổ phiếu cần kiểm tra
ma_cophieu = "VCB"  

# Khởi tạo giá trị nhỏ nhất cho phần trăm thay đổi
min_percent_change = float('inf')
min_record = None

# Truy vấn lịch sử giá của mã cổ phiếu
lich_su_gia = collection.find_one({"lich_su_gia."+ma_cophieu: {"$exists": True}}, {"lich_su_gia."+ma_cophieu: 1})

if lich_su_gia:
    for record in lich_su_gia["lich_su_gia"][ma_cophieu]:
        percent_change_str = record.get("percent_change", "").replace(",", "").replace("%", "")
        
        # Chỉ chuyển đổi nếu giá trị là số hợp lệ
        if percent_change_str and percent_change_str != "-":
            try:
                percent_change = float(percent_change_str)
                if percent_change < min_percent_change:
                    min_percent_change, min_record = percent_change, record
            except ValueError:
                continue

if min_record:
    print(f"Percent change nhỏ nhất cho mã {ma_cophieu} là {min_percent_change}%, thông tin: {min_record}")
else:
    print(f"Không tìm thấy dữ liệu cho mã cổ phiếu {ma_cophieu}.")


Percent change nhỏ nhất cho mã VCB là -0.98%, thông tin: {'date': '20/09/2024', 'opening_price': '91,800', 'highest_price': '92,000', 'lowest_price': '90,600', 'closing_price': '90,600', 'price_change': '-900', 'percent_change': '-0.98%', 'volume': '2,180,400'}


In [53]:
#9 in ra sự chêch lệch khối lượng giữa 2 data có ngày gần nhất trong một cỗ phiếu

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Mã cổ phiếu cần truy vấn
ma_cophieu = "TPB"  

# Lấy lịch sử giá của cổ phiếu
lich_su_gia = collection.find_one({}, {"lich_su_gia": 1}).get("lich_su_gia", {}).get(ma_cophieu, [])

# Sắp xếp theo ngày giảm dần
lich_su_gia.sort(key=lambda x: datetime.strptime(x["date"], "%d/%m/%Y"), reverse=True)

# Tính chênh lệch khối lượng giữa 2 ngày gần nhất
if len(lich_su_gia) >= 2:
    volume_difference = (
        int(lich_su_gia[0]["volume"].replace(",", "")) - 
        int(lich_su_gia[1]["volume"].replace(",", ""))
    )
    print(f"Sự chênh lệch khối lượng giữa 2 ngày gần nhất ({lich_su_gia[0]['date']} và {lich_su_gia[1]['date']}): {volume_difference}")
else:
    print("Không đủ dữ liệu để tính toán sự chênh lệch khối lượng.")


Không đủ dữ liệu để tính toán sự chênh lệch khối lượng.


In [54]:
#10 in ra sự chêch lệch khối lượng giữa 2 data có ngày bất kì trong một cỗ phiếu

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Mã cổ phiếu và ngày cần so sánh
ma_cophieu = "VCB"
date1 = "10/10/2024"  # Ngày thứ nhất
date2 = "25/10/2024"  # Ngày thứ hai

# Lấy lịch sử giá của mã cổ phiếu
lich_su_gia = collection.find_one({"linhvuc": "Tài chính ngân hàng"}, {"lich_su_gia": 1})

# Khởi tạo biến khối lượng
volumes = {date1: None, date2: None}

if lich_su_gia and ma_cophieu in lich_su_gia["lich_su_gia"]:
    records = lich_su_gia["lich_su_gia"][ma_cophieu]
    
    # Tìm khối lượng cho cả 2 ngày
    for record in records:
        record_date = record["date"]
        if record_date in volumes:
            volumes[record_date] = int(record["volume"].replace(",", ""))

    # Tính và in ra sự chênh lệch khối lượng
    if None not in volumes.values():
        difference = abs(volumes[date1] - volumes[date2])
        print(f"Sự chênh lệch khối lượng giữa {date1} và {date2} cho mã {ma_cophieu} là: {difference}")
    else:
        print("Không tìm thấy dữ liệu cho một trong hai ngày đã chỉ định.")
else:
    print(f"Không tìm thấy dữ liệu cho mã cổ phiếu {ma_cophieu}.")


Sự chênh lệch khối lượng giữa 10/10/2024 và 25/10/2024 cho mã VCB là: 328900


In [55]:
#11 lấy khối lượng lớn nhất cho hai mã bất kì và tính sự chêch lẹch khối lượng

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Mã cổ phiếu bạn muốn truy vấn
ma_cophieu_1 = "VCB"  # Mã cổ phiếu thứ nhất
ma_cophieu_2 = "BID"  # Mã cổ phiếu thứ hai

# Hàm để lấy khối lượng cao nhất
def get_highest_volume(ma_cophieu):
    lich_su_gia = collection.find_one({}, {"lich_su_gia": 1}).get("lich_su_gia", {}).get(ma_cophieu, [])
    return max((int(record["volume"].replace(",", "")) for record in lich_su_gia), default=None)

# Lấy khối lượng cao nhất cho hai mã cổ phiếu
highest_volume_1 = get_highest_volume(ma_cophieu_1)
highest_volume_2 = get_highest_volume(ma_cophieu_2)

# In kết quả và tính chênh lệch
if highest_volume_1 is not None and highest_volume_2 is not None:
    print(f"Khối lượng cao nhất cho mã {ma_cophieu_1}: {highest_volume_1}")
    print(f"Khối lượng cao nhất cho mã {ma_cophieu_2}: {highest_volume_2}")

    difference = abs(highest_volume_1 - highest_volume_2)
    if highest_volume_1 > highest_volume_2:
        print(f"Khối lượng cao nhất của {ma_cophieu_1} cao hơn {ma_cophieu_2} với sự chênh lệch là {difference}.")
    elif highest_volume_1 < highest_volume_2:
        print(f"Khối lượng cao nhất của {ma_cophieu_2} cao hơn {ma_cophieu_1} với sự chênh lệch là {difference}.")
    else:
        print(f"Khối lượng cao nhất của hai mã {ma_cophieu_1} và {ma_cophieu_2} bằng nhau.")
else:
    print("Không đủ dữ liệu để tính toán.")


Không đủ dữ liệu để tính toán.


In [56]:
#12 xuất tên các mã có dữ liệu từ ngày nào tới ngày nào

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Ngày bắt đầu và ngày kết thúc
start_date = datetime.strptime("2/10/2024", "%d/%m/%Y")  
end_date = datetime.strptime("2/10/2024", "%d/%m/%Y")    

# Lưu trữ các mã có dữ liệu trong khoảng thời gian đã cho
ma_cophieu_with_data = set()  # Sử dụng set để loại bỏ mã trùng lặp

# Lấy tất cả mã cổ phiếu
for document in collection.find({}, {"lich_su_gia": 1}):
    lich_su_gia = document.get("lich_su_gia", {})
    for ma_cophieu, records in lich_su_gia.items():
        # Kiểm tra từng bản ghi trong khoảng thời gian
        if any(start_date <= datetime.strptime(record["date"], "%d/%m/%Y") <= end_date for record in records):
            ma_cophieu_with_data.add(ma_cophieu)

# In ra tên các mã có dữ liệu
print(f"Các mã có dữ liệu từ {start_date.strftime('%d/%m/%Y')} đến {end_date.strftime('%d/%m/%Y')}:")
print(ma_cophieu_with_data)  # In ra set trực tiếp


Các mã có dữ liệu từ 02/10/2024 đến 02/10/2024:
{'VCB'}


In [57]:
#13 in ra dữ liệu của 3 mã bất kì

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Chỉ định ba mã cổ phiếu mà bạn muốn tìm
ma_cophieu_list = ["VCB", "BID", "CTG"]

# Lấy lịch sử giá cho từng mã cổ phiếu
lich_su_gia = collection.find_one({"linhvuc": "Tài chính ngân hàng"}, {"lich_su_gia": 1})

if lich_su_gia:
    for ma_cophieu in ma_cophieu_list:
        records = lich_su_gia["lich_su_gia"].get(ma_cophieu, [])

        if records:
            print(f"Dữ liệu cho mã cổ phiếu {ma_cophieu}:")
            for record in records:
                print(record)
            print("\n")  # Thêm một dòng trống giữa các mã cổ phiếu
        else:
            print(f"Không tìm thấy dữ liệu cho mã cổ phiếu {ma_cophieu}.")
else:
    print("Không tìm thấy dữ liệu cho lĩnh vực tài chính ngân hàng.")


Dữ liệu cho mã cổ phiếu VCB:
{'date': '30/10/2024', 'opening_price': '92,000', 'highest_price': '92,400', 'lowest_price': '91,600', 'closing_price': '91,700', 'price_change': '-300', 'percent_change': '-0.33%', 'volume': '626,500'}
{'date': '29/10/2024', 'opening_price': '92,100', 'highest_price': '92,600', 'lowest_price': '92,000', 'closing_price': '92,000', 'price_change': '-', 'percent_change': '-', 'volume': '1,297,600'}
{'date': '28/10/2024', 'opening_price': '91,500', 'highest_price': '92,300', 'lowest_price': '91,500', 'closing_price': '92,000', 'price_change': '+200', 'percent_change': '0.22%', 'volume': '1,105,800'}
{'date': '25/10/2024', 'opening_price': '92,100', 'highest_price': '92,300', 'lowest_price': '91,700', 'closing_price': '91,800', 'price_change': '+100', 'percent_change': '0.11%', 'volume': '996,300'}
{'date': '24/10/2024', 'opening_price': '92,200', 'highest_price': '92,300', 'lowest_price': '91,400', 'closing_price': '91,700', 'price_change': '+200', 'percent_ch

In [58]:
#14 Tính giá trung bình đóng cửa của một mã cổ phiếu trong khoảng thời gian nhất định

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Chỉ định mã cổ phiếu và khoảng thời gian
ma_cophieu = "VCB"  # Mã cổ phiếu bạn muốn tính
start_date_str = "01/10/2024"  # Ngày bắt đầu
end_date_str = "31/10/2024"    # Ngày kết thúc

# Chuyển đổi ngày sang định dạng datetime
start_date = datetime.strptime(start_date_str, "%d/%m/%Y")
end_date = datetime.strptime(end_date_str, "%d/%m/%Y")

# Lấy lịch sử giá cho mã cổ phiếu
lich_su_gia = collection.find_one({"linhvuc": "Tài chính ngân hàng"}, {"lich_su_gia": 1})

if lich_su_gia:
    records = lich_su_gia["lich_su_gia"].get(ma_cophieu, [])
    total_closing_price = 0
    count = 0

    # Tính giá trung bình đóng cửa
    for record in records:
        record_date = datetime.strptime(record["date"], "%d/%m/%Y")
        if start_date <= record_date <= end_date:
            closing_price = float(record["closing_price"].replace(",", ""))
            total_closing_price += closing_price
            count += 1

    # In ra kết quả
    if count > 0:
        average_closing_price = total_closing_price / count
        print(f"Giá trung bình đóng cửa của mã {ma_cophieu} từ {start_date_str} đến {end_date_str} là: {average_closing_price:.2f}")
    else:
        print(f"Không có dữ liệu cho mã {ma_cophieu} trong khoảng thời gian đã chỉ định.")
else:
    print(f"Không tìm thấy dữ liệu cho mã cổ phiếu {ma_cophieu}.")


Giá trung bình đóng cửa của mã VCB từ 01/10/2024 đến 31/10/2024 là: 91840.91


In [59]:
#15 Tìm mã cổ phiếu có sự thay đổi giá cao nhất trong một khoảng thời gian

# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['simplize']
collection = db['Tài chính ngân hàng']

# Chỉ định khoảng thời gian
start_date_str = "01/10/2024"  # Ngày bắt đầu
end_date_str = "31/10/2024"    # Ngày kết thúc

# Chuyển đổi ngày sang định dạng datetime
start_date = datetime.strptime(start_date_str, "%d/%m/%Y")
end_date = datetime.strptime(end_date_str, "%d/%m/%Y")

# Tìm mã cổ phiếu có sự thay đổi giá cao nhất
max_price_change = float('-inf')
max_ma_cophieu = None

# Lấy tất cả các mã cổ phiếu trong collection
linhvuc_data = collection.find({"linhvuc": "Tài chính ngân hàng"})  # Lưu ý: có thể cần chỉnh sửa tên linh vực

for linhvuc in linhvuc_data:
    lich_su_gia = linhvuc.get("lich_su_gia", {})
    for ma_cophieu, records in lich_su_gia.items():
        for record in records:
            record_date = datetime.strptime(record["date"], "%d/%m/%Y")
            if start_date <= record_date <= end_date:
                percent_change_str = record.get("percent_change", "").replace(",", "").replace("%", "")
                if percent_change_str and percent_change_str != '-':  # Kiểm tra giá trị
                    try:
                        percent_change = float(percent_change_str)
                        if percent_change > max_price_change:
                            max_price_change = percent_change
                            max_ma_cophieu = ma_cophieu
                    except ValueError:
                        continue  # Bỏ qua nếu không thể chuyển đổi

if max_ma_cophieu:
    print(f"Mã cổ phiếu có sự thay đổi giá cao nhất từ {start_date_str} đến {end_date_str} là: {max_ma_cophieu} với sự thay đổi là: {max_price_change:.2f}%")
else:
    print("Không tìm thấy dữ liệu trong khoảng thời gian đã chỉ định.")


Mã cổ phiếu có sự thay đổi giá cao nhất từ 01/10/2024 đến 31/10/2024 là: VCB với sự thay đổi là: 0.99%
